In [0]:
# synthetic_dataset.py
#
# Creates a toy data set that satisfies the structure required by
# optimisation_model.py.  Feel free to tweak the constants at the
# top to make the instance larger or more challenging.

import random
import numpy as np

# ------------------------------------------------------------------
# 0 — controls for reproducibility & size
# ------------------------------------------------------------------
RNG_SEED       = 42         # change for a new random instance
N_TIER1        = 5          # finished products
N_TIER2        = 10          # sub-assemblies
N_TIER3        = 20          # raw parts / first-tier suppliers
PART_TYPES     = ['a', 'b', 'c', 'd']   # bill-of-material codes
DISRUPT_FRACTION = 0.15     # share of nodes to mark as disrupted
random.seed(RNG_SEED)
np.random.seed(RNG_SEED)

# ------------------------------------------------------------------
# 1 — elementary node sets
# ------------------------------------------------------------------
tier1      = [f'T1_{i}' for i in range(1, N_TIER1 + 1)]
tier2      = [f'T2_{i}' for i in range(1, N_TIER2 + 1)]
tier3      = [f'T3_{i}' for i in range(1, N_TIER3 + 1)]
part_types = PART_TYPES.copy()

# nodes chosen to be disrupted in this scenario
n_disrupted = max(1, int(DISRUPT_FRACTION * (N_TIER2 + N_TIER3)))
disrupted   = random.sample(tier2 + tier3, n_disrupted)

# ------------------------------------------------------------------
# 2 — link-structure: N_minus, P, N_plus
# ------------------------------------------------------------------
N_minus = {}                       # j  → list of part types k
P       = {}                       # (j, k) → list of parent nodes i
N_plus  = {n: [] for n in tier2 + tier3}   # i → list of children j

def _connect(children, parents, max_parts=2, max_parents=2):
    """Populate N_minus, P and N_plus so that every `child` consumes
    1…max_parts part types, each supplied by 1…max_parents `parent` nodes."""
    for j in children:
        needed_parts = random.sample(part_types,
                                     random.randint(1, max_parts))
        N_minus[j] = needed_parts
        for k in needed_parts:
            chosen_parents = random.sample(parents,
                                           random.randint(1, max_parents))
            P[(j, k)] = chosen_parents
            for i in chosen_parents:
                N_plus[i].append(j)

# tier-1 products are assembled from tier-2 sub-assemblies
_connect(tier1, tier2, max_parts=2, max_parents=2)
# tier-2 sub-assemblies are made from tier-3 raw parts
_connect(tier2, tier3, max_parts=1, max_parents=2)

# tier-3 nodes are leaves (need no parts themselves)
for j in tier3:
    N_minus[j] = []

# Make sure every node in U (tier2+tier3) appears in N_plus, even if empty
for n in tier2 + tier3:
    N_plus.setdefault(n, [])

# ------------------------------------------------------------------
# 3 — numerical parameters
# ------------------------------------------------------------------
# profit margin per finished product
f = {j: round(float(np.random.uniform(0.05, 0.30)), 2) for j in tier1}

# on-hand inventory for every node
s = {n: int(np.random.randint(0, 51)) for n in tier1 + tier2 + tier3}

# demand per TTR for each finished product
d = {j: int(np.random.randint(400, 1801)) for j in tier1}

# production capacity per TTR for every node
c = {n: int(np.random.randint(500, 2201)) for n in tier1 + tier2 + tier3}

# time-to-recover for this scenario (scalar)
t = 1

# ------------------------------------------------------------------
# 4 — quick sanity check (run `python synthetic_dataset.py` directly)
# ------------------------------------------------------------------
if __name__ == "__main__":
    print(">>> Elementary sets")
    print("tier1 :", tier1)
    print("tier2 :", tier2)
    print("tier3 :", tier3)
    print("part_types :", part_types)
    print("disrupted :", disrupted)
    print("\n>>> One sample of N_minus / P / N_plus")
    j_sample = tier1[0]
    print(f"N_minus[{j_sample}] =", N_minus[j_sample])
    for k in N_minus[j_sample]:
        print(f"  P[({j_sample}, {k})] → {P[(j_sample, k)]}")
    i_sample = tier2[0]
    print(f"N_plus[{i_sample}] =", N_plus[i_sample])
    print("\n>>> Parameter snippets")
    print("f :", f)
    print("d :", d)
    print("c :", {k: c[k] for k in tier1[:2]})   # first two for brevity
    print("s :", {k: s[k] for k in tier1[:2]})

# ------------------------------------------------------------------
# Data objects are now defined *at module level* and can be imported:
#
#   from synthetic_dataset import *
#   from optimisation_model import main    # if you wrapped solve-block in main()
#
# or simply execute optimisation_model.py in the same interpreter session.
# ------------------------------------------------------------------


In [0]:
# draw_supply_network.py
#
# Visualise the 3-tier supply network created in synthetic_dataset.py.
# Run with:  python draw_supply_network.py
# ---------------------------------------------------------------

import matplotlib.pyplot as plt
import networkx as nx

# ---------------------------------------------------------------
# 1 build a directed graph  (edge i ➔ j means “i supplies j”)
# ---------------------------------------------------------------
G = nx.DiGraph()
G.add_nodes_from(tier1 + tier2 + tier3)

for parent, children in N_plus.items():
    for child in children:
        G.add_edge(parent, child)

# ---------------------------------------------------------------
# 2 layout: put tiers on horizontal stripes (top: T3  → bottom: T1)
# ---------------------------------------------------------------
pos = {}
y_gap = 1.5          # vertical spacing between tiers
x_gap = 2.0          # horizontal spacing within each tier

def _place(nodes, y, x_start=0.0):
    for k, n in enumerate(nodes):
        pos[n] = (x_start + k * x_gap, y)

_place(tier1, y=2 * y_gap)               # finished products (top)
_place(tier2, y=1 * y_gap)               # sub-assemblies
_place(tier3, y=0 * y_gap)               # raw parts  (bottom)

# ---------------------------------------------------------------
# 3 style: colours by tier, arrows, nice labels
# ---------------------------------------------------------------
colour_map = []
for n in G.nodes:
    if n in tier1:
        colour_map.append("#ec7063")     # red   – finished products
    elif n in tier2:
        colour_map.append("#5dade2")     # blue  – sub-assemblies
    else:
        colour_map.append("#58d68d")     # green – raw parts

plt.figure(figsize=(10, 6))
nx.draw_networkx_edges(G, pos, arrowstyle="->", arrowsize=12, width=1.2)
nx.draw_networkx_nodes(G, pos, node_color=colour_map, node_size=800, edgecolors="k")
nx.draw_networkx_labels(G, pos, font_size=10, font_family="sans-serif", font_weight="bold")

plt.axis("off")
plt.title("Synthetic 3-Tier Supply Network", fontsize=14, pad=20)
plt.tight_layout()
plt.show()